<a href="https://colab.research.google.com/github/hmghaly/word_align/blob/master/bert_walign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
cwd='/content/drive/MyDrive/tr_ar' #directory where we keep the data
if not os.path.exists(cwd): os.makedirs(cwd)  #if the directory doesn't exist, create it
os.chdir(cwd) #change current work directory to the directory where we keep the data
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/tr_ar


In [2]:
#Clone the github codebase
!rm -r code_utils #first delete the code_utils folder, then clone from the codebase (very crude till we figure out using github from colab)
!git clone https://github.com/hmghaly/word_align.git code_utils

Cloning into 'code_utils'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 237 (delta 29), reused 0 (delta 0), pack-reused 186
Receiving objects: 100% (237/237), 922.90 KiB | 3.68 MiB/s, done.
Resolving deltas: 100% (130/130), done.


In [3]:
#For parsing functions
import spacy
from code_utils.dep_lib import *

nlp = spacy.load("en_core_web_sm")
nlp.tokenizer=nlp.tokenizer.tokens_from_list
#print(dir(nlp.tokenizer))
def get_conll(sent_input):
  #nlp = spacy.load("en_core_web_sm")
  #if type(sent_input) is list: nlp.tokenizer=nlp.tokenizer.tokens_from_list
  doc = nlp(sent_input)
  conll_2d=[]
  for token in doc:	
    token_i = token.i+1
    if token.i==token.head.i: head_i=0
    else: head_i = token.head.i+1
    items=[token_i,token.text, token.lemma_, token.tag_, token.pos_, "_", head_i, token.dep_,"_","_"]
    conll_2d.append(items)
  return conll_2d



text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
text="she went to school"
text="At its forty - seventh session , after considering the report of the Special Committee , 3 / the General Assembly adopted resolution 47 / 23 of 25 November 1992 , by which it , inter alia :"
text="At its forty - sixth session , the General Assembly , by adopting resolution 46 / 181 of 19 December 1991 , endorsed as a Plan of Action for the International Decade for the Eradication of Colonialism , the proposals contained in the annex to the report of the Secretary - General dated 13 December 1991 ( A/46/634/Rev.1 ) . The Plan , inter alia , contained the following provisions :"
text="The Special Committee on the Situation with regard to the Implementation of the Declaration on the Granting of Independence to Colonial Countries and Peoples , with the cooperation of the administering Powers , should :"
text="On the occasion of the tenth , twentieth , twenty - fifth and thirtieth anniversaries of the Declaration on the Granting of Independence to Colonial Countries and Peoples , the General Assembly , by approving the related reports of the Special Committee , adopted resolutions 2621 ( XXV ) of 12 October 1970 , 35 / 118 of 11 December 1980 , 40 / 56 of 2 December 1985 and 45 / 33 of 20 November 1990 , containing a series of recommendations with a view to facilitating the speedy implementation of the Declaration ."
words=text.split()
cur_conll_2d=get_conll(words)
cur_conll_str="\n".join(["\t".join([str(it) for it in v]) for v in cur_conll_2d]) 
conll_obj=conll(cur_conll_str)
for ap in conll_obj.all_phrases:
  print(ap)
# for key,val in conll_obj.child_dict.items():
#   print(key,val)
# child_dict={}
# for dd in conll_obj.dep_dict:
#   all_children=recursive_dep(dd,conll_obj.dep_dict,[])
#   all_children=sorted(list(set(all_children)))
#   child_dict[dd]=all_children
#   #print(dd,all_children)
# phrase_label_counter={}
# all_phrases=[]
# for c2 in cur_conll_2d:
#   cur_ind,cur_word,cur_pos,cur_head=c2[0],c2[1],c2[3],c2[6]
#   # cur_depth=conll_obj.basic_dep_dict.get(cur_ind,0)
#   # print("cur_depth:",cur_depth)
#   direct_dependendents=conll_obj.dep_dict.get(cur_ind,[])
#   cur_children=child_dict.get(cur_ind,[])
#   if cur_children==[]: min_child_i,max_child_i=cur_ind,cur_ind
#   else: min_child_i,max_child_i=min(cur_children),max(cur_children)

#   #give an ID to the current word, based on its POS
#   if not cur_pos[0].isalnum(): cur_pos="PUNCT"
#   if cur_pos.startswith("NN"): cur_pos="NN"
#   cur_phrase_count=phrase_label_counter.get(cur_pos,0)
#   cur_pos_id=cur_pos+str(cur_phrase_count+1)
#   phrase_label_counter[cur_pos]=cur_phrase_count+1

#   #give the phrase label to its projections
#   #print(cur_ind,cur_word,cur_pos,cur_head,direct_dependendents,[min_child_i,max_child_i])
#   if cur_pos[0] in "VN": phrase_label=cur_pos[0].upper()+"P"
#   elif cur_pos=="JJ": phrase_label="AP"
#   elif cur_pos=="IN": phrase_label="PP"
#   elif cur_pos=="MD": phrase_label="VP"
#   else: phrase_label="XP"

#   dependents_before=sorted([v for v in direct_dependendents if v<cur_ind],reverse=True)
#   dependents_after=[v for v in direct_dependendents if v>cur_ind]
#   prejections=dependents_after+dependents_before
#   #print(cur_ind,cur_word,cur_pos,cur_head,prejections)


#   #print(cur_pos_id,cur_ind,[cur_ind])
#   tmp_phrase_obj={}
#   tmp_phrase_obj["id"]=cur_pos_id
#   tmp_phrase_obj["head"]=cur_ind
#   tmp_phrase_obj["terminal"]=True
#   tmp_phrase_obj["range"]=(cur_ind,cur_ind)
#   tmp_phrase_obj["text"]=cur_word
#   tmp_phrase_obj["n"]=1

#   all_phrases.append(tmp_phrase_obj)
  

#   all_children_so_far=[cur_ind]
#   for pr in prejections:
#     cur_phrase_count=phrase_label_counter.get(phrase_label,0)
#     cur_phrase_id=phrase_label+str(cur_phrase_count+1)
#     phrase_label_counter[phrase_label]=cur_phrase_count+1
#     #attached_projections.append(pr)
#     tmp_children_of_the_attached_phrase=child_dict.get(pr,[pr])
#     all_children_so_far.extend(list(tmp_children_of_the_attached_phrase))
#     tmp_children_so_far=sorted(list(all_children_so_far))
#     phrase_range=(min(tmp_children_so_far),max(tmp_children_so_far))
#     words=[conll_obj.word_dict[v] for v in tmp_children_so_far]
#     #print(cur_phrase_id, pr,tmp_children_of_the_attached_phrase,phrase_range)
#     #print(">>>",cur_phrase_id, phrase_range,  )
#     tmp_phrase_obj={}
#     tmp_phrase_obj["id"]=cur_phrase_id
#     tmp_phrase_obj["head"]=cur_ind
#     tmp_phrase_obj["terminal"]=False
#     tmp_phrase_obj["range"]=phrase_range
#     tmp_phrase_obj["text"]=" ".join(words)
#     tmp_phrase_obj["n"]=len(words)
#     all_phrases.append(tmp_phrase_obj)
#     #print("-----")
    
#     # cur_children_indexes=child_dict.get(pr,[pr])
#     # print(pr,cur_children_indexes)
#     # cur_projection_children_indexes.extend(cur_children_indexes)
#     # all_projections_with_children.append(list(cur_projection_children_indexes))
#     # words=[conll_obj.word_dict[v] for v in list(cur_projection_children_indexes)]
#     # print(pr)
#     # print(cur_phrase_id,words)


#   #print(cur_ind,cur_word,cur_pos,cur_head,direct_dependendents,"children range:",[min_child_i,max_child_i])

#   # for ap in all_projections_with_children:
#   #   print(ap)

#   #print("=================")
# for ap in all_phrases:
#   print(ap)
# for ind in conll_obj.index_list:
#   cur_ind,cur_wd=ind
#   cur_children=child_dict.get(cur_ind,[])
#   print(ind, conll_obj.pos_dict[cur_ind],cur_children)




{'id': 'IN1', 'head': 1, 'terminal': True, 'range': (0, 0), 'text': 'On', 'n': 1}
{'id': 'PP1', 'head': 1, 'terminal': False, 'range': (0, 31), 'text': 'On the occasion of the tenth , twentieth , twenty - fifth and thirtieth anniversaries of the Declaration on the Granting of Independence to Colonial Countries and Peoples , the General Assembly', 'n': 32}
{'id': 'DT1', 'head': 2, 'terminal': True, 'range': (1, 1), 'text': 'the', 'n': 1}
{'id': 'NN1', 'head': 3, 'terminal': True, 'range': (2, 2), 'text': 'occasion', 'n': 1}
{'id': 'NP1', 'head': 3, 'terminal': False, 'range': (2, 22), 'text': 'occasion of the tenth , twentieth , twenty - fifth and thirtieth anniversaries of the Declaration on the Granting of Independence', 'n': 21}
{'id': 'NP2', 'head': 3, 'terminal': False, 'range': (2, 31), 'text': 'occasion of the tenth , twentieth , twenty - fifth and thirtieth anniversaries of the Declaration on the Granting of Independence to Colonial Countries and Peoples , the General Assembly',

# Now Install Awsome Align

In [4]:
!pip install transformers==3.1.0
import torch
import transformers
import itertools

     |████████████████████████████████| 890kB 12.4MB/s 
     |████████████████████████████████| 890kB 25.9MB/s 
     |████████████████████████████████| 3.0MB 47.7MB/s 
     |████████████████████████████████| 1.2MB 38.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=5055df60d457f4109cb4ffadc66fbe57fb8e9aa73007c7b95f31179ce27cc4e9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [9]:
model = transformers.BertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')
def bert_walign(src,tgt):
  sent_src, sent_tgt = src.strip().split(), tgt.strip().split()
  token_src, token_tgt = [tokenizer.tokenize(word) for word in sent_src], [tokenizer.tokenize(word) for word in sent_tgt]
  wid_src, wid_tgt = [tokenizer.convert_tokens_to_ids(x) for x in token_src], [tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
  ids_src, ids_tgt = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids'], tokenizer.prepare_for_model(list(itertools.chain(*wid_tgt)), return_tensors='pt', truncation=True, model_max_length=tokenizer.model_max_length)['input_ids']
  sub2word_map_src = []
  for i, word_list in enumerate(token_src):
    sub2word_map_src += [i for x in word_list]
  sub2word_map_tgt = []
  for i, word_list in enumerate(token_tgt):
    sub2word_map_tgt += [i for x in word_list]

  # alignment
  align_layer = 8
  threshold = 1e-3
  model.eval()
  with torch.no_grad():
    out_src = model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
    out_tgt = model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]

    dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))

    softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
    softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)

    softmax_inter = (softmax_srctgt > threshold)*(softmax_tgtsrc > threshold)

  align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
  align_words = set()
  for i, j in align_subwords:
    align_words.add( (sub2word_map_src[i], sub2word_map_tgt[j]) )
  return sorted(align_words)

# printing
class color:
  PURPLE = '\033[95m'
  CYAN = '\033[96m'
  DARKCYAN = '\033[36m'
  BLUE = '\033[94m'
  GREEN = '\033[92m'
  YELLOW = '\033[93m'
  RED = '\033[91m'
  BOLD = '\033[1m'
  UNDERLINE = '\033[4m'
  END = '\033[0m'

# for i, j in sorted(align_words):
#   print(i,j,f'{color.BOLD}{color.BLUE}{sent_src[i]}{color.END}==={color.BOLD}{color.RED}{sent_tgt[j]}{color.END}')  

In [39]:
import os
from code_utils.align_utils import *
from code_utils.arabic_lib import *
from code_utils.general import *
from itertools import groupby
from collections import Counter

ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"

start_i=0
n=None
max_phrase_size=12


phrase_table_fname="phrase_table.txt"

# phrase_table_fopen=open(phrase_table_fname,"w") #to reset, if needed
# phrase_table_fopen.close()

if os.path.exists(phrase_table_fname):
  phrase_table_fopen=open(phrase_table_fname)
  for line in phrase_table_fopen:
    line_split=line.split("\t")
    print(line_split)
    if len(line_split)>1: start_i=int(line_split[0])+1
  phrase_table_fopen.close()




ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)

src_sents_tok,trg_sents_tok=[],[]
for sent_i,pair in enumerate(zip(en_fopen,ar_fopen)):
  if sent_i<start_i: continue
  if n!=None and sent_i>=start_i+n: break
  if sent_i%20000==0: print(sent_i)
  phrase_table_fopen=open(phrase_table_fname,"a")

  en_line,ar_line=pair
  en_line,ar_line=en_line.strip("\n\r\t "),ar_line.strip("\n\r\t ")

  en_toks,ar_toks=tok(en_line),tok(ar_line)

  #en_toks=[v.lower() for v in en_toks]
  ar_toks=tok_ar(ar_toks,ar_counter_dict)

  en_str,ar_str=" ".join([v.lower() for v in en_toks])," ".join(ar_toks)
  
  ar_str=ar_str.replace("_","")
  print(en_str)
  print(ar_str)
  
  aligned_pairs=bert_walign(en_str,ar_str)
  aligned_trg_indexes=list(set([v[1] for v in aligned_pairs]))
  conll_out=get_conll(en_toks)
  cur_conll_str="\n".join(["\t".join([str(it) for it in v]) for v in conll_out]) 
  conll_obj=conll(cur_conll_str)
  for ap in conll_obj.all_phrases:
    if ap["n"]>max_phrase_size: continue
    if ap["id"][0] in "NJ" or "P" in ap["id"]: 
      phrase_type=ap["id"].strip("0123456789")
      #print(ap)  #print(conll_out)
      src_phrase=ap["text"]
      src_phrase_i0,src_phrase_i1=ap["range"]
      trg_phrase_j0,trg_phrase_j1=len(ar_toks),0
      for i0,j0 in aligned_pairs:
        
        if i0<src_phrase_i0 or i0>src_phrase_i1: continue
        #print(src_phrase_i0,src_phrase_i1, i0,j0)
        if j0<trg_phrase_j0: trg_phrase_j0=j0
        if j0>trg_phrase_j1: trg_phrase_j1=j0
      trg_phrase=" ".join(ar_toks[trg_phrase_j0:trg_phrase_j1+1])
      if trg_phrase=="": continue
      #now we want to exclude phrases one of the points have the y coordinate in the span we just identified and the x outside
      accept=True
      for i0,j0 in aligned_pairs:
        if j0<trg_phrase_j0 or j0>trg_phrase_j1: continue
        if i0<src_phrase_i0 or i0>src_phrase_i1: 
          accept=False
          break
      if not accept:
        print("unacceptable >>>>", sent_i, src_phrase, trg_phrase)


      trg_starting_indexes=[trg_phrase_j0]
      trg_ending_indexes=[trg_phrase_j1]
      for c0 in range(1,7): #we check the unaligned points to the left
        if trg_phrase_j0-c0 in aligned_trg_indexes: break
        if trg_phrase_j0-c0<0: break
        trg_starting_indexes.append(trg_phrase_j0-c0)
      for c1 in range(1,7): #we check the unaligned points to the right
        if trg_phrase_j1+c1 in aligned_trg_indexes: break
        if trg_phrase_j1+c1>=len(ar_toks): break
        trg_ending_indexes.append(trg_phrase_j1+c1)


      #print(src_phrase)
      #print(">>>>", trg_phrase )
      for tmp_j0 in trg_starting_indexes:
        for tmp_j1 in trg_ending_indexes:
          tmp_trg_phrase=" ".join(ar_toks[tmp_j0:tmp_j1+1])
          #print(phrase_type, src_phrase, tmp_trg_phrase)
          cur_line="%s\t%s\t%s\t%s\n"%(sent_i,phrase_type, src_phrase, tmp_trg_phrase)
          phrase_table_fopen.write(cur_line)
      #print("--------")
      



    

  #print("===============")
  phrase_table_fopen.close()
ar_fopen.close()
en_fopen.close()


Streaming output truncated to the last 5000 lines.
VP observe the requirements of article 15 ( 1 ) شروط ال_مادة 15 ) 1 ( ، يذكر
VP observe the requirements of article 15 ( 1 ) بـ شروط ال_مادة 15 ) 1 ( ، يذكر
--------
VP To observe the requirements of article 15 ( 1 ) تقيدا بـ شروط ال_مادة 15 ) 1 ( ، يذكر
VP To observe the requirements of article 15 ( 1 ) وـ تقيدا بـ شروط ال_مادة 15 ) 1 ( ، يذكر
--------
NP requirements of article 15 ( 1 ) شروط ال_مادة 15 ) 1
NP requirements of article 15 ( 1 ) شروط ال_مادة 15 ) 1 (
NP requirements of article 15 ( 1 ) بـ شروط ال_مادة 15 ) 1
NP requirements of article 15 ( 1 ) بـ شروط ال_مادة 15 ) 1 (
--------
NP the requirements of article 15 ( 1 ) شروط ال_مادة 15 ) 1
NP the requirements of article 15 ( 1 ) شروط ال_مادة 15 ) 1 (
NP the requirements of article 15 ( 1 ) بـ شروط ال_مادة 15 ) 1
NP the requirements of article 15 ( 1 ) بـ شروط ال_مادة 15 ) 1 (
--------
PP of article 15 ( 1 ) شروط ال_مادة 15 ) 1
PP of article 15 ( 1 ) شروط ال_مادة 15 ) 1 (
PP 

KeyboardInterrupt: ignored

In [ ]:
import spacy
from code_utils.dep_lib import *
nlp = spacy.load("en_core_web_sm")
nlp.tokenizer=nlp.tokenizer.tokens_from_list
#print(dir(nlp.tokenizer))
def get_conll(sent_input):
  #nlp = spacy.load("en_core_web_sm")
  #if type(sent_input) is list: nlp.tokenizer=nlp.tokenizer.tokens_from_list
  doc = nlp(sent_input)
  conll_2d=[]
  for token in doc:	
    token_i = token.i+1
    if token.i==token.head.i: head_i=0
    else: head_i = token.head.i+1
    items=[token_i,token.text, token.lemma_, token.tag_, token.pos_, "_", head_i, token.dep_,"_","_"]
    conll_2d.append(items)
  return conll_2d

class conll1: #we will use this one, so we can convert to PTB at some point
	def __init__(self,conll_str,pos_index=3,dep_index=6,drel_index=7,full_analysis=True): #full analysis to include syntactic constituents, phonological/prosodic chunks, and also configurations
		self.dep_index=dep_index
		self.drel_index=drel_index
		self.dep_dict={} #depedency dict; which indexes depend on a given head
		self.index_head_dict={} #what is the head of each given index
		self.word_dict={} #this index corresponds to this word
		self.pos_dict={} #this index corresponds to this POS
		self.deprel_dict={} #the dependency relation of this index
		self.head_offset_dict={} #the distance between each word and its head
		self.index_list=[] #the list of indexes and their corresponding words
		self.full_list=[] #the list of indexes and their corresponding words, POS and dependencies
		self.conll_2d=[v.split("\t") for v in conll_str.split("\n") if v] #splitting the conll string into 2-d array
		self.dep_index_pairs=[]
		self.size=len(self.conll_2d)
		self.all_heads=[]
		self.all_deps=[] #index-head_index pairs
		self.all_words=[]
		self.all_indexes=[]
		self.all_head_offsets=[]
		self.basic_dep_dict={}
		
		

		for c2 in self.conll_2d:
			cur_index,depends_on,cur_dep_rel=int(c2[0]),int(c2[dep_index]),c2[drel_index]
			word,pos=c2[1],c2[pos_index]
			self.index_list.append((cur_index,word))
			self.full_list.append((cur_index,word,pos,depends_on)) #depends on should be head_index
			self.word_dict[cur_index]=word
			self.pos_dict[cur_index]=pos
			self.deprel_dict[cur_index]=cur_dep_rel
			self.dep_dict[depends_on]=self.dep_dict.get(depends_on,[])+[cur_index]
			self.dep_index_pairs.append((cur_index,depends_on)) #this is the one we need: pairs of indexes, heads
			if depends_on==0: head_offset=0
			else: head_offset=cur_index-depends_on
			self.head_offset_dict[pos_index]=head_offset
			self.all_heads.append(depends_on)
			if depends_on!=0: self.all_deps.append((cur_index,depends_on)) #we will need to adjust the naming so all_deps doesn't exclude roots
			self.index_head_dict[cur_index]=depends_on
			self.all_words.append(word)
			self.all_indexes.append(cur_index)
			self.all_head_offsets.append(head_offset)
			self.basic_dep_dict[cur_index]=depends_on

	self.child_dict={} #recursively get all children and descendents
	for dd in self.dep_dict:
		all_children=recursive_dep(dd,self.dep_dict,[])
		all_children=sorted(list(set(all_children)))
		self.child_dict[dd]=all_children
	phrase_label_counter={}
	self.all_phrases=[]
	for c2 in self.conll_2d:
		cur_ind,cur_word,cur_pos,cur_head=int(c2[0]),c2[1],c2[3],int(c2[6])
		direct_dependendents=self.dep_dict.get(cur_ind,[])
	  #print("cur_ind,cur_word,cur_pos,cur_head",cur_ind,cur_word,cur_pos,cur_head)  
		print("cur_ind,direct_dependendents",cur_ind,self.dep_dict.get(cur_ind,[]))        
		cur_children=self.child_dict.get(cur_ind,[])
		if cur_children==[]: min_child_i,max_child_i=cur_ind,cur_ind
		else: min_child_i,max_child_i=min(cur_children),max(cur_children)

	  #give an ID to the current word, based on its POS
		if not cur_pos[0].isalnum(): cur_pos="PUNCT"
		if cur_pos.startswith("NN"): cur_pos="NN"
		cur_phrase_count=phrase_label_counter.get(cur_pos,0)
		cur_pos_id=cur_pos+str(cur_phrase_count+1)
		phrase_label_counter[cur_pos]=cur_phrase_count+1

	  #give the phrase label to its projections
	  #print(cur_ind,cur_word,cur_pos,cur_head,direct_dependendents,[min_child_i,max_child_i])
		if cur_pos[0] in "VN": phrase_label=cur_pos[0].upper()+"P"
		elif cur_pos=="JJ": phrase_label="AP"
		elif cur_pos=="IN": phrase_label="PP"
		elif cur_pos=="MD": phrase_label="VP"
		else: phrase_label="XP"

		dependents_before=sorted([v for v in direct_dependendents if v<cur_ind],reverse=True)
		dependents_after=[v for v in direct_dependendents if v>cur_ind]
		prejections=dependents_after+dependents_before


	  #print(cur_pos_id,cur_ind,[cur_ind])
		tmp_phrase_obj={}
		tmp_phrase_obj["id"]=cur_pos_id
		tmp_phrase_obj["head"]=cur_ind
		tmp_phrase_obj["terminal"]=True
		tmp_phrase_obj["range"]=(cur_ind,cur_ind)
		tmp_phrase_obj["text"]=cur_word
		tmp_phrase_obj["n"]=1
		self.all_phrases.append(tmp_phrase_obj)
	  

		all_children_so_far=[cur_ind]

		for pr in prejections:
			cur_phrase_count=phrase_label_counter.get(phrase_label,0)
			cur_phrase_id=phrase_label+str(cur_phrase_count+1)
			phrase_label_counter[phrase_label]=cur_phrase_count+1
			tmp_children_of_the_attached_phrase=self.child_dict.get(pr,[pr])
			all_children_so_far.extend(list(tmp_children_of_the_attached_phrase))
			tmp_children_so_far=sorted(list(all_children_so_far))
			phrase_range=(min(tmp_children_so_far),max(tmp_children_so_far))
			print("tmp_children_so_far",tmp_children_so_far,self.word_dict)
			words=[self.word_dict[v] for v in tmp_children_so_far]
			tmp_phrase_obj={}
			tmp_phrase_obj["id"]=cur_phrase_id
			tmp_phrase_obj["head"]=cur_ind
			tmp_phrase_obj["terminal"]=False
			tmp_phrase_obj["range"]=phrase_range
			tmp_phrase_obj["text"]=" ".join(words)
			tmp_phrase_obj["n"]=len(words)
			self.all_phrases.append(tmp_phrase_obj)



		self.word_dep_pairs=[(self.word_dict.get(v[0],""),self.word_dict.get(v[1],"")) for v in self.dep_index_pairs]
		self.word_deprel_pairs=[(self.word_dict.get(v[0],""),self.deprel_dict.get(v[0],"")) for v in self.dep_index_pairs]
		self.word_deprel_word_triplets=[(self.word_dict.get(v[0],""),self.deprel_dict.get(v[0],""),self.word_dict.get(v[1],"")) for v in self.dep_index_pairs]
		self.word_pos_pairs=[(self.word_dict.get(v[0],""),self.pos_dict.get(v[0],"")) for v in self.dep_index_pairs]

		#advanced processing of conll to get further info from the structure

		#syntactic phrase geometry information
		self.phrase_list=[]
		self.number_phrases_end_index_list=[0]*self.size #number of phrases ending at a certain index
		self.number_phrases_start_index_list=[0]*self.size #number of phrases starting *immediately after* the current index
		self.size_phrases_end_index_list=[0]*self.size #size of the largest phrase ending at each index
		self.size_phrases_start_index_list=[0]*self.size #size of the largest phrase starting *after* the current index

		#depth of each word within the structure (key: word index, value, depth)
		self.depth_dict={}
		self.list_depths=[]

		#phonological/prosodic dependency chunks, obtained by grouping each head with its adjacent non-head children 
		self.dep_chunks=[] #the chunks of words themselves
		self.chunk_boundaries=[0]*self.size #list of indexes, indicating whether each index correspond to chunk boundary or not

		#dependency configuration
		self.list_dep_configs_raw=[] #list of dependency configurations between each two consecutive words, with the original head offsets -6,+2
		self.list_dep_configs=[] #same list but normalized -1,+1
		self.list_link_configs=[] #number of leftward links, rightward links, and overhead links between each two consecutive words

text="The Special Committee on the Situation with regard to the Implementation of the Declaration on the Granting of Independence to Colonial Countries and Peoples , with the cooperation of the administering Powers , should :"
words=text.split()
cur_conll_2d=get_conll(words)
cur_conll_str="\n".join(["\t".join([str(it) for it in v]) for v in cur_conll_2d])
ob1=conll1(cur_conll_str)
for ap in ob1.all_phrases:
  print(ap)
print(ob1.dep_dict)


NameError: ignored

In [ ]:
ob1.word_dict

{1: 'The',
 2: 'Special',
 3: 'Committee',
 4: 'on',
 5: 'the',
 6: 'Situation',
 7: 'with',
 8: 'regard',
 9: 'to',
 10: 'the',
 11: 'Implementation',
 12: 'of',
 13: 'the',
 14: 'Declaration',
 15: 'on',
 16: 'the',
 17: 'Granting',
 18: 'of',
 19: 'Independence',
 20: 'to',
 21: 'Colonial',
 22: 'Countries',
 23: 'and',
 24: 'Peoples',
 25: ',',
 26: 'with',
 27: 'the',
 28: 'cooperation',
 29: 'of',
 30: 'the',
 31: 'administering',
 32: 'Powers',
 33: ',',
 34: 'should',
 35: ':'}